In [5]:
import findspark
from pyspark.sql import SparkSession
import schemas
from test import most_used_tags

findspark.init()


def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark, sc


def main():
    spark, sc = init_spark()
    timestamp_format = 'YYYY-MM-DD HH:mm:ss z'
    dummy_switch = '-1.4.0-2018-12-22'
    # dummy_switch = '_dummy'

    projects_path = f'Libraries.io-open-data-1.4.0/libraries/projects{dummy_switch}.csv'
    projects = spark.read.csv(projects_path, header=True, timestampFormat=timestamp_format,
                          schema=schemas.projects)
    most_used_tags(projects)
    # nums = sc.parallelize([1, 2, 3, 4])
    # print(nums.map(lambda x: x * x).collect())
    # dist_file = sc.textFile('Libraries.io-open-data-1.4.0.tar.gz', 5000)
    # size = dist_file.map(lambda s: len(s)).reduce(lambda a, b: a+b)
    # print(size)


if __name__ == '__main__':
    main()



[Row(Keyword='react', count=36788), Row(Keyword='api', count=24728), Row(Keyword='javascript', count=23676), Row(Keyword='library', count=21238), Row(Keyword='laravel', count=20495), Row(Keyword='framework', count=19733), Row(Keyword='angular', count=18990), Row(Keyword='plugin', count=18054), Row(Keyword='test', count=17573), Row(Keyword='css', count=16400)]


In [16]:
spark, sc = init_spark()
import pyspark.sql.functions as F
dependencies_path = f'Libraries.io-open-data-1.4.0/libraries/dependencies-1.4.0-2018-12-22.csv'
projects_path = f'Libraries.io-open-data-1.4.0/libraries/projects-1.4.0-2018-12-22.csv'
timestamp_format = 'YYYY-MM-DD HH:mm:ss z'

dependencies = spark.read.csv(projects_path, header=True, schema=schemas.projects)
dependencies = spark.read.csv(dependencies_path, header=True, schema=schemas.dependencies).drop('ID').select('Project ID', 'Dependency Project ID')
    dependency_pairs = dependencies.where(F.col('Dependency Project ID').isNotNull()).limit(10)
    pageranks = dependency_pairs \
        .select('Project ID') \
        .union(dependency_pairs
               .select('Dependency Project ID')) \
        .distinct().alias('ID') \
        .withColumn('pagerank', F.lit(1))
    dependency_counts = dependency_pairs.groupBy('Project ID').count()
    # print(pageranks.count(), pageranks.take(5))
    for it in range(2):
        print(f'It {it}')
        dependency_pairs = pageranks \
            .join(dependency_pairs, 'Project ID') \
            .join(dependency_counts, 'Project ID') \
            .withColumn('vote', F.col('pagerank') / F.col('count'))\
            .drop('pagerank', 'count')
        vote_sums = dependency_pairs\
            .groupBy('Dependency Project ID')\
            .sum('vote')\
            .withColumnRenamed('sum(vote)', 'vote_sum')
        # print(vote_sums.take(5))
        pageranks = pageranks\
            .join(vote_sums, pageranks['Project ID'] == vote_sums['Dependency Project ID'], how='left')\
            .drop('pagerank')\
            .drop('Dependency Project ID')\
            .withColumn('pagerank', 1 + F.col('vote_sum'))\
            .drop('vote_sum') \
            .fillna({'pagerank': 1})
        pageranks.show(10)
    pageranks.join(projects, pageranks['Project ID']==projects['I'])

[('C#', ['Newtonsoft.Json', 'EntityFramework', 'Microsoft.AspNet.Mvc', 'Microsoft.AspNet.Razor', 'Antlr', 'Microsoft.Net.Compilers', 'Microsoft.Owin', 'Owin', 'NUnit', 'Microsoft.Owin.Host.SystemWeb']), ('JavaScript', ['express', 'lodash', 'debug', 'body-parser', 'async', 'babel-core', 'ansi-regex', 'ansi-styles', 'chalk', 'balanced-match']), ('C++', ['nan', 'node-sass', 'bser', 'fb-watchman', 'therubyracer', 'unf_ext', 'bcrypt', 'uws', 'electron', 'facter']), ('Julia', ['Compat', 'Distributions', 'DataFrames', 'DataStructures', 'JSON', 'StatsBase', 'BinDeps', 'Reexport', 'PyCall', 'Colors']), ('C', ['ffi', 'fsevents', 'rb-fsevent', 'binding_of_caller', 'bcrypt', 'nio4r', 'numpy', 'bcrypt-ruby', 'psycopg2', 'redcarpet']), ('Objective-C', ['AFNetworking', 'SDWebImage', 'expo', 'MBProgressHUD', 'Masonry', 'cordova-plugin-splashscreen', 'babel-preset-expo', 'SVProgressHUD', 'Bolts', 'MJRefresh']), ('Swift', ['Alamofire', 'SwiftyJSON', 'ReactiveCocoa', 'RxSwift', 'SnapKit', 'Kingfisher', '